In [27]:
import psycopg2
import os
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

conn = psycopg2.connect(
    host=os.getenv("POSTGRES_SERVER").split(":")[0],
    port=os.getenv("POSTGRES_SERVER").split(":")[1],
    user=os.getenv("POSTGRES_USERNAME"),
    password=os.getenv("POSTGRES_PASSWORD"),
    database=os.getenv("POSTGRES_DATABASE")
)

In [28]:
cur = conn.cursor()
cur.execute("""
    SELECT table_name 
    FROM information_schema.tables
    WHERE table_schema = 'public'
    ORDER BY table_name;
""")
# cur.execute("""
#     SELECT pid,
#        state,
#        wait_event_type,
#        wait_event,
#        query
# FROM pg_stat_activity
# WHERE datname = 'patientproj2_0v1';
# """)

tables = cur.fetchall()
df = pd.DataFrame(tables, columns=["Table Name"])
df

,Table Name
0,age_groups
1,anime
2,anime_genres
3,anime_statuses
4,anime_types
5,countries
6,genders
7,genres
8,pg_stat_statements
9,pg_stat_statements_info


In [29]:
cur = conn.cursor()
cur.execute("SELECT * FROM anime LIMIT 5;")
rows = cur.fetchall()
colnames = [desc[0] for desc in cur.description]
df = pd.DataFrame(rows, columns=colnames)
df


,anime_id,title,type_id,status_id,episodes,start_date,end_date,source_id,studio_id,rating_category_id,overall_score,popularity_rank
0,A001,Fullmetal Alchemist: Brotherhood,2,1,64,2009-04-05,2010-07-04,4,11,1,9.15,1
1,A002,Attack on Titan,2,1,75,2013-04-07,2021-03-29,4,37,1,9.08,2
2,A003,One Piece,2,2,1100,1999-10-20,NaT,7,1,2,8.80,3
3,A004,Demon Slayer: Kimetsu no Yaiba,2,1,26,2019-04-06,2019-09-28,4,14,1,8.85,4
4,A005,Jujutsu Kaisen,2,2,24,2020-10-03,NaT,4,37,1,8.75,5


In [30]:
cur.execute("""
    SELECT anime_id, title, episodes, overall_score
    FROM anime
    ORDER BY popularity_rank
    LIMIT 5;
""")
rows = cur.fetchall()
colnames = [desc[0] for desc in cur.description]
df = pd.DataFrame(rows, columns=colnames)
df

,anime_id,title,episodes,overall_score
0,A001,Fullmetal Alchemist: Brotherhood,64,9.15
1,A002,Attack on Titan,75,9.08
2,A003,One Piece,1100,8.80
3,A004,Demon Slayer: Kimetsu no Yaiba,26,8.85
4,A005,Jujutsu Kaisen,24,8.75


In [31]:
cur = conn.cursor()
cur.execute("""
    SELECT 
        a.anime_id,
        a.title,
        at.type_name AS type,
        ast.status_desc AS status,
        s.studio_name,
        a.overall_score,
        a.popularity_rank
    FROM anime a
    LEFT JOIN anime_types at ON at.type_id = a.type_id
    LEFT JOIN anime_statuses ast ON ast.status_id = a.status_id
    LEFT JOIN studios s ON s.studio_id = a.studio_id
    ORDER BY a.popularity_rank
    LIMIT 10;
""")
rows = cur.fetchall()
colnames = [desc[0] for desc in cur.description]
df = pd.DataFrame(rows, columns=colnames)
df


,anime_id,title,type,status,studio_name,overall_score,popularity_rank
0,A001,Fullmetal Alchemist: Brotherhood,TV,Finished,Bones,9.15,1
1,A002,Attack on Titan,TV,Finished,MAPPA,9.08,2
2,A003,One Piece,TV,Ongoing,Toei Animation,8.80,3
3,A004,Demon Slayer: Kimetsu no Yaiba,TV,Finished,Ufotable,8.85,4
4,A005,Jujutsu Kaisen,TV,Ongoing,MAPPA,8.75,5
5,A006,Steins;Gate,TV,Finished,White Fox,9.05,6
6,A007,Death Note,TV,Finished,Madhouse,8.70,7
7,A008,Fullmetal Alchemist (2003),TV,Finished,Bones,8.15,8
8,A009,Hunter x Hunter (2011),TV,Finished,Madhouse,9.00,9
9,A010,My Hero Academia,TV,Ongoing,Bones,8.10,10


In [32]:
cur = conn.cursor()
cur.execute("""
    SELECT 
        r.user_id,
        u.user_name,
        a.title,
        r.user_score,
        r.rating_date,
        ws.status_desc AS watch_status
    FROM user_anime_ratings r
    JOIN users u ON u.user_id = r.user_id
    JOIN anime a ON a.anime_id = r.anime_id
    JOIN watch_statuses ws ON ws.watch_status_id = r.watch_status_id
    LIMIT 10;
""")
rows = cur.fetchall()
colnames = [desc[0] for desc in cur.description]
df = pd.DataFrame(rows, columns=colnames)
df


,user_id,user_name,title,user_score,rating_date,watch_status
0,U001,anime_fan_001,Demon Slayer: Kimetsu no Yaiba,7.0,2024-04-29,Completed
1,U001,anime_fan_001,Violet Evergarden,8.0,2024-06-25,Completed
2,U001,anime_fan_001,Tokyo Ghoul,8.0,2024-05-22,On-Hold
3,U001,anime_fan_001,Your Lie in April,6.0,2024-01-16,On-Hold
4,U001,anime_fan_001,Gintama,7.0,2024-12-23,Completed
5,U001,anime_fan_001,Re:Zero − Starting Life in Another World,10.0,2024-05-22,On-Hold
6,U001,anime_fan_001,Konosuba 2,9.0,2024-04-22,Dropped
7,U001,anime_fan_001,Overlord III,7.0,2024-11-28,Dropped
8,U001,anime_fan_001,Chihayafuru,6.0,2024-07-13,Completed
9,U001,anime_fan_001,Fire Force Season 2,8.0,2024-01-23,Dropped
